# **AI3001: Advanced Topics in Machine Learning**
Homework 2 - Question 5 <br>
Name- Pushkal Mishra <br>
Roll- EE20BTECH11042 <br>

## **Importing all the necessary libraries**

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

## **Declaring some constants**

In [ ]:
all_K = [10, 20, 50, 100]
all_T = [1000, 10000, 50000, 100000, 500000, 1000000, 5000000]
T_map = {1000 : 0, 10000 : 1, 50000 : 2, 100000 : 3, 500000 : 4, 1000000 : 5, 5000000 : 6}
runs = 500

## **For EXP3-γ**

In [ ]:
error = []
exp3_regret = []

for K in all_K:

  mu = np.zeros(K, dtype = np.float32)
    
  for i in range(K):
    mu[i] = 1 - (i + 1) / (K + 1)
    
  indexes = np.arange(K)
  regret = np.zeros(len(all_T))

  for T in all_T:
 
    gamma = ((K * math.log(K)) / (2 * T)) ** 0.5
    c1 = gamma / K
    c2 = c1 * np.ones(K, dtype = np.float32)

    for run in range(runs):

      reg = 0
      weights = np.ones(K, dtype = np.float32)
    
      for t in range(T):
        
        if (t % 1000) == 0:
          print(f"K: {K}  T: {T}  Run: {run + 1}  t: {t + 1}")
        
        weights_sum = np.sum(weights)
        
        c3 = (1 - gamma) / weights_sum
        
        p_t = c2 + (c3 * weights)
        sum_p_t = np.sum(p_t)

        if np.isnan(sum_p_t):
          # regret[T_map[T]] -= reg
          error.append(f"Run: {run + 1}  T: {T}  t: {t + 1}")
          break

        p_t /= sum_p_t
        i_t = np.random.choice(indexes, size = 1, p = p_t)
        r_i_t = np.random.binomial(1, mu[i_t])
        r_i_t_hat = r_i_t / p_t[i_t]
        r_i_star = np.random.binomial(1, mu[0])
        
        weights[i_t] *= math.exp(c1 * r_i_t_hat)

        reg += (r_i_star - r_i_t)
        regret[T_map[T]] += (r_i_star - r_i_t)

    regret[T_map[T]] /= runs
  exp3_regret.append(regret)

In [ ]:
legend = ["K = 10", "K = 20", "K = 50", "K = 100"]
plt.xlabel("Time Horizon T")
plt.ylabel("Regret")
plt.title("Using EXP3-γ algorithm")
plt.xscale("log")
plt.yscale("log")
for i in range(len(exp3_regret)):
  plt.plot(all_T, exp3_regret[i], marker = 'o')
plt.legend(legend)
plt.show()

## **For exploration separated algorithm**

In [ ]:
exploration_regret = []

for K in all_K:
  
  mu = np.zeros(K, dtype = np.float32)
    
  for i in range(K):
    mu[i] = 1 - (i + 1) / (K + 1)
    
  regret = np.zeros(len(all_T))

  for T in all_T:

    epsilon = T ** (-1.0 / 3.0)
    exploration = int((epsilon * T) / K) * K

    for run in range(runs):

      S = np.zeros(K, dtype = np.float32)
      N = np.zeros(K, dtype = np.float32)
      
      for t in range(exploration):

        i_t = (t % K)
        N[i_t] += 1

        r_i_t = np.random.binomial(1, mu[i_t])
        r_i_star = np.random.binomial(1, mu[0])
        
        S[i_t] = S[i_t] + r_i_t
        regret[T_map[T]] += (r_i_star - r_i_t)

      mu_hat = S / N
      mu_max_value = max(mu_hat)
      lst = mu_hat.tolist()
      i_mu_max = lst.index(mu_max_value)

      for t in range(T - exploration):

        r_i_mu_max = np.random.binomial(1, mu[i_mu_max])
        r_i_star = np.random.binomial(1, mu[0])
        regret[T_map[T]] += (r_i_star - r_i_t)

    regret[T_map[T]] /= runs
  
  exploration_regret.append(regret)

In [ ]:
legend = ["K = 10", "K = 20", "K = 50", "K = 100"]
plt.xlabel("Time Horizon T")
plt.ylabel("Regret")
plt.title("Using Exploration-separated algorithm")
plt.xscale("log")
# plt.yscale("log")
for i in range(len(exploration_regret)):
  plt.plot(all_T, exploration_regret[i], marker = 'o')
plt.legend(legend)
plt.show()

## **For UCB-1**

In [ ]:
ucb1_regret = []

for K in all_K:

  mu = np.zeros(K, dtype = np.float32)
    
  for i in range(K):
    mu[i] = 1 - (i + 1) / (K + 1)
    
  regret = np.zeros(len(all_T))

  for T in all_T:

    for run in range(runs):

      ucb_scores = np.zeros(K, dtype = np.float32)
      num_visited = np.zeros(K)

      for t in range(K):
        
        r_i_1 = np.random.binomial(1, mu[t])
        num_visited[t] += 1
        mu_hat = r_i_1 / num_visited[t]

        ucb_scores[t] = mu_hat + np.sqrt((2 * np.log(t + 1)) / num_visited[t])

        r_i_star = np.random.binomial(1, mu[0])
        regret[T_map[T]] += (r_i_star - r_i_1)
      
      for t in range(T - K):

        max_ucb_score = max(ucb_scores)
        lst = ucb_scores.tolist()
        i_t = lst.index(max_ucb_score)
        
        r_i_t = np.random.binomial(1, mu[i_t])
        num_visited[i_t] += 1
        mu_hat_i_t = r_i_t / num_visited[i_t]

        ucb_scores[i_t] = mu_hat_i_t + np.sqrt((2 * np.log(K + t + 1)) / num_visited[i_t])

        r_i_star = np.random.binomial(1, mu[0])
        regret[T_map[T]] += (r_i_star - r_i_t)
    
    regret[T_map[T]] /= runs
  
  ucb1_regret.append(regret)

In [ ]:
legend = ["K = 10", "K = 20", "K = 50", "K = 100"]
plt.xlabel("Time Horizon T")
plt.ylabel("Regret")
plt.title("Using UCB-1 algorithm")
plt.xscale("log")
# plt.yscale("log")
for i in range(len(ucb1_regret)):
  plt.plot(all_T, ucb1_regret[i], marker = 'o')
plt.legend(legend)
plt.show()

## **For Thompson sampling**

In [ ]:
thompson_regret = []

for K in all_K:

  mu = np.zeros(K, dtype = np.float32)

  for i in range(K):
    mu[i] = 1 - (i + 1) / (K + 1)
    
  regret = np.zeros(len(all_T))

  for T in all_T:

    for run in range(runs):

      S = np.zeros(K, dtype = np.float32)
      F = np.zeros(K, dtype = np.float32)
      all_lambda = np.zeros(K, dtype = np.float32)

      for t in range(T):

        for i in range(K):

          all_lambda[i] = np.random.beta(S[i] + 1, F[i] + 1)
        
        max_lambda = max(all_lambda)
        lst = all_lambda.tolist()
        i_t = lst.index(max_lambda)

        r_i_t = np.random.binomial(1, mu[i_t])
        r_i_star = np.random.binomial(1, mu[0])
        regret[T_map[T]] += (r_i_star - r_i_t)

        S[i_t] += r_i_t
        F[i_t] += (1 - r_i_t)
    
    regret[T_map[T]] /= runs
  
  thompson_regret.append(regret)

In [ ]:
legend = ["K = 10", "K = 20", "K = 50", "K = 100"]
plt.xlabel("Time Horizon T")
plt.ylabel("Regret")
plt.title("Using Thompson sampling")
plt.xscale("log")
for i in range(len(thompson_regret)):
  plt.plot(all_T, thompson_regret[i], marker = 'o')
plt.legend(legend)
plt.show()